In [124]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [125]:
import pandas as pd
import numpy as np
import joblib

In [126]:
ROOT = '/content/drive/Othercomputers/Mi MacBook Pro/Google Drive/tp2/'
dataSetPath = './data/smoking_prediction_entrega.csv'

df = pd.read_csv(ROOT + dataSetPath)
df_copy = df.copy()

In [127]:
df.drop(columns=['ID'], axis=1, inplace=True)

In [128]:
drop_columns = ['oral', 'eyesight(left)', 'eyesight(right)', 'hearing(left)', 'hearing(right)', 'serum creatinine']
df.drop(columns=drop_columns, axis=1, inplace=True)

In [129]:
df['gender'] = df['gender'].astype('category')
df['gender'] = df['gender'].cat.codes

In [130]:
df['tartar'] = df['tartar'].astype('category')
df['tartar'] = df['tartar'].cat.codes

In [131]:
X = df.values

In [132]:
pipe = joblib.load(ROOT + 'models/pipeline.joblib')

In [133]:
predict = pipe.predict(X)

In [134]:
df_copy['smooking_predict'] = predict

In [135]:
df_copy

,ID,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),...,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,oral,dental caries,tartar,smooking_predict
0,27358,M,25,160,65,3.42,0.05,0.00,0.04,0.04,...,0.63,0.04,0.01,0.75,0.71,0.71,Y,0,Y,0
1,27364,M,30,180,80,3.46,0.04,0.01,0.04,0.04,...,0.59,0.04,0.04,0.79,1.13,1.33,Y,0,N,1
2,27368,M,55,165,60,3.42,0.00,0.01,0.04,0.04,...,0.63,0.04,0.01,1.08,1.29,2.00,Y,1,Y,1
3,27378,M,20,175,75,3.63,0.05,0.05,0.04,0.04,...,0.63,0.04,0.04,0.83,0.58,0.46,Y,0,N,1
4,27381,M,25,165,80,3.79,0.04,0.04,0.04,0.04,...,0.67,0.04,0.04,1.25,1.63,1.96,Y,1,Y,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5687,55676,F,40,170,65,3.13,0.01,0.01,0.04,0.04,...,0.50,0.04,0.00,0.58,0.29,0.42,Y,1,Y,0
5688,55681,F,45,160,50,2.92,0.04,0.04,0.04,0.04,...,0.58,0.04,0.01,0.83,0.50,0.58,Y,0,Y,0
5689,55683,F,55,160,50,2.84,0.04,0.04,0.04,0.04,...,0.50,0.04,0.00,0.71,0.46,0.50,Y,0,N,0
5690,55684,M,60,165,60,3.25,0.01,0.04,0.04,0.04,...,0.59,0.04,0.00,0.83,0.79,0.75,Y,0,N,1


In [136]:
df_copy.to_csv(ROOT + './predictions/smoking_prediction_resolved.csv')
np.savetxt(ROOT + './predictions/predictions.txt', predict.astype(int), fmt="%d")
np.savetxt(ROOT + './predictions/predictions.csv', predict.astype(int), fmt="%d", delimiter=",")